In [1]:
#include<mlpack/xeus-cling.hpp>
#include<mlpack/core.hpp>
#include<mlpack/core/data/split_data.hpp>
#include<mlpack/methods/ann/layer/layer.hpp>
#include<mlpack/methods/ann/ffn.hpp>
#include<bits/stdc++.h>
#include<ensmallen.hpp>

In [2]:
using namespace mlpack;


In [3]:
using namespace mlpack::ann;

In [4]:
using namespace arma;

In [5]:
using namespace std;

In [6]:
using namespace ens;

In [7]:
arma::Row<size_t> getLabels(arma::mat predOut){
    arma::Row<size_t> predLabels(predOut.n_cols);
    for( arma::uword i = 0; i < predOut.n_cols; ++i){
        predLabels(i) = predOut.col(i).index_max();
    }
    return predLabels;
}

In [8]:
constexpr double RATIO = 0.1;
constexpr int MAX_ITERATIONS = 0;
constexpr double STEP_SIZE = 1.2e-3;
constexpr int BATCH_SIZE = 50;

In [9]:
arma::mat dataset;

In [10]:
data::Load("/home/viole/swaingotnochill/examples/generating_hand_written_digits_mnist_with_gan/digit-recognizer/train.csv", dataset, true);

In [11]:
arma::mat train, valid;

In [12]:
data::Split(dataset, train, valid, RATIO);

In [13]:
const arma::mat trainX = train.submat(1, 0, train.n_rows - 1, train.n_cols - 1);
const arma::mat validX = valid.submat(1, 0, valid.n_rows - 1, valid.n_cols - 1);

In [14]:
const arma::mat trainY = train.row(0);
const arma::mat validY = valid.row(0);

In [15]:
FFN<NegativeLogLikelihood<>, RandomInitialization> model;

In [16]:
model.Add<mlpack::ann::Convolution<>> (1,6,5,5,1,1,0,0,28,28);
                        

In [17]:
model.Add<LeakyReLU<>>();

In [18]:
model.Add<MaxPooling<>>(2,
                       2,
                       2,
                       2,
                       true);

In [19]:
model.Add<Convolution<>>(6,
                        16,
                        5,
                        5,
                        1,
                        1,
                        0,
                        0,
                        12,
                        12);

In [20]:
model.Add<LeakyReLU<>>();

In [21]:
model.Add<LeakyReLU<>>();

In [22]:
model.Add<MaxPooling<>>(2,2,2,2,true);

In [23]:
model.Add<Linear<>>(16*4*4, 10);

In [24]:
model.Add<LogSoftMax<>>();

In [25]:
ens::Adam optimizer(STEP_SIZE,
                   BATCH_SIZE,
                   0.9,
                   0.999,
                   1e-8,
                   MAX_ITERATIONS,
                   1e-8,
                   true);

In [26]:
model.Train(trainX,
           trainY,
           optimizer,
           ens::PrintLoss(),
           ens::ProgressBar(),
           ens::EarlyStopAtMinLoss(
           [&](const arma::mat&){
               double validationLoss = model.Evaluate(validX, validY);
               std::cout << "Validation Loss" << validationLoss << "." << "\n";
               return validationLoss;
           }))


error: Mat::operator(): index out of bounds


Standard Exception: Mat::operator(): index out of bounds